In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu
from suite3d import nbtools as nbui
from suite3d import extension as ext

/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/transport.py:258: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [34]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'AH012',
    'date' :          '2024-08-10',
    'expnum' :         [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21]}

tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=False)

job_params = {    
    # number of channels recorded in the tiff file, typically 30
    'n_ch_tif' : 26,
    # number of planes in the deeper cavity, typically 15
    'cavity_size' : 13,
    # convert from the Scanimage channel ordering to deep-to-shallow ordering
    # in our case, ScanImage channel numbers are in temporal order. 
    'planes' : n.array([ 0,  2,  4,  6,  8, 10, 12, 14,
                        16, 18, 20, 22, 24, 1,  3, 5,  7,9,11, 13, 15,17,19,21])[:22],
    # number of files to use for the initial pass
    'n_init_files' :   4,
    
    # number of pixels to fuse between the ROI strips
    # the auto-detection doesn't always work well, recommend 
    # manually tuning it to reduce the stitching artifacts
    'fuse_shift_override' : 6,
    
    # will try to automatically estimate crosstalk using 
    # the shallowest crosstalk_n_planes planes. if you want to override,
    # set override_crosstalk = float between 0 and 1
    'subtract_crosstalk' : True,
    
    # volume rate in acquisition
    'fs' : flu.get_si_params(tifs[0])['vol_rate'],
    
    # 3D GPU registration - fast! 
    # for now, 3D + GPU are well-tested, other options might have minor bugs
    # if you run into them, let us know! we'll fix them
    '3d_reg' : True,
    'nonrigid' : False,
    'gpu_reg' : True,
    'save_dtype' : 'float16',
}

In [37]:

job_params['fs'] = si_params['vol_rate']

savedir = os.path.join('/mnt/md0/runs', expt_info['subject'], expt_info['date'])
os.makedirs(savedir, exist_ok=True)

# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,
          params=job_params, create=True, overwrite=True, verbosity = 3)

Job directory /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21 already exists
Loading job directory for AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21 in /mnt/md0/runs
      Created dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/iters
   Loading default params
      Updating param n_ch_tif
      Updating param cavity_size
      Updating param planes
      Updating param n_init_files
      Updating param fuse_shift_override
      Updating param subtract_crosstalk
      Updating param fs
      Updating param 3d_reg
      Updating param nonrigid
      Updating param gpu_reg
      Updating param save_dtype
   Updated main params file


In [38]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 4
job.params['init_n_frames'] = None
job.params['init_file_sample_method'] = 'even'
job.params['voxel_size_um'] = (20, 2.5, 2.5)

In [39]:
# %%time
# summary = job.run_init_pass()

In [40]:
summary = job.load_summary()

In [41]:
img = summary['ref_img_3d']

In [42]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [43]:
img.shape

(22, 1040, 741)

In [44]:
%%time
job.register()

      Found dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/registered_fused_data
      Updating self.dirs tag registered_fused_data
   Saved a copy of params at /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/registered_fused_data
   Updated main params file
   Starting registration: 3D: True, GPU: True
   Overriding fuse shift value to 6
   Not clipping frames for registration
Will analyze 546 tifs in 546 batches
   Enforcing positivity
   Launching IO thread
         Memory at batch 0.  Total Used: 103.280 GB, Virtual Available: 415.601 GB, Virtual Used: 087.594 GB, Swap Used: 015.686 GB
Loading Batch 0 of 545
      Loading /mnt/zortex-subjects/AH012/2024-08-10/1/2024-08-10_1_AH012_2P_00001_00001.tif
    Loaded file into shared memory in 33.97 sec


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

    Workers completed in 3.16 sec
    Total time: 37.13 sec
   Loaded 1 files, total 2.95 GB
   Batch 0 IO thread joined
         Memory after IO thread joinTotal Used: 106.268 GB, Virtual Available: 412.613 GB, Virtual Used: 090.583 GB, Swap Used: 015.686 GB
         Memory after movie copied from threadTotal Used: 109.225 GB, Virtual Available: 409.657 GB, Virtual Used: 093.539 GB, Swap Used: 015.686 GB
         Memory after thread memory clearedTotal Used: 105.961 GB, Virtual Available: 412.920 GB, Virtual Used: 090.276 GB, Swap Used: 015.686 GB
   Launching IO thread for next batch
         After IO thread launch:Total Used: 105.961 GB, Virtual Available: 412.920 GB, Virtual Used: 090.276 GB, Swap Used: 015.686 GB
      Loaded batch of size (22, 100, 1000, 720)
      Loading /mnt/zortex-subjects/AH012/2024-08-10/1/2024-08-10_1_AH012_2P_00001_00002.tif
   Completed rigid reg on batch in :7.1043853759765625s
   Shifted the mov in: 2.0932602882385254s
         After all GPU Batches:To

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

    Workers completed in 5.45 sec
    Total time: 38.45 sec
   Loaded 1 files, total 2.95 GB
   Batch 1 IO thread joined
         Memory after IO thread joinTotal Used: 119.743 GB, Virtual Available: 399.138 GB, Virtual Used: 104.057 GB, Swap Used: 015.686 GB
         Memory after movie copied from threadTotal Used: 116.367 GB, Virtual Available: 402.514 GB, Virtual Used: 100.682 GB, Swap Used: 015.686 GB
         Memory after thread memory clearedTotal Used: 113.416 GB, Virtual Available: 405.465 GB, Virtual Used: 097.731 GB, Swap Used: 015.686 GB
   Launching IO thread for next batch
         After IO thread launch:Total Used: 113.416 GB, Virtual Available: 405.465 GB, Virtual Used: 097.731 GB, Swap Used: 015.686 GB
      Loading /mnt/zortex-subjects/AH012/2024-08-10/1/2024-08-10_1_AH012_2P_00001_00003.tif
      Loaded batch of size (22, 100, 1000, 720)
   Completed rigid reg on batch in :6.636544942855835s
   Shifted the mov in: 1.9823968410491943s
         After all GPU Batches:Tot

...

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

    Workers completed in 5.26 sec
    Total time: 40.09 sec
   Loaded 1 files, total 2.95 GB
   Batch 142 IO thread joined
         Memory after IO thread joinTotal Used: 120.896 GB, Virtual Available: 397.986 GB, Virtual Used: 105.210 GB, Swap Used: 015.686 GB
         Memory after movie copied from threadTotal Used: 117.520 GB, Virtual Available: 401.361 GB, Virtual Used: 101.835 GB, Swap Used: 015.686 GB
         Memory after thread memory clearedTotal Used: 114.561 GB, Virtual Available: 404.320 GB, Virtual Used: 098.876 GB, Swap Used: 015.686 GB
   Launching IO thread for next batch
         After IO thread launch:Total Used: 114.561 GB, Virtual Available: 404.320 GB, Virtual Used: 098.876 GB, Swap Used: 015.686 GB
      Loaded batch of size (22, 100, 1000, 720)
      Loading /mnt/zortex-subjects/AH012/2024-08-10/5/2024-08-10_5_AH012_2P_00001_00022.tif
   Completed rigid reg on batch in :6.717633962631226s
   Shifted the mov in: 1.930727243423462s
         After all GPU Batches:To

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

    Workers completed in 5.35 sec
    Total time: 37.06 sec
   Loaded 1 files, total 2.95 GB
   Batch 143 IO thread joined
         Memory after IO thread joinTotal Used: 120.239 GB, Virtual Available: 398.642 GB, Virtual Used: 104.554 GB, Swap Used: 015.686 GB
         Memory after movie copied from threadTotal Used: 116.869 GB, Virtual Available: 402.012 GB, Virtual Used: 101.184 GB, Swap Used: 015.686 GB
         Memory after thread memory clearedTotal Used: 113.913 GB, Virtual Available: 404.969 GB, Virtual Used: 098.227 GB, Swap Used: 015.686 GB
   Launching IO thread for next batch
         After IO thread launch:Total Used: 113.913 GB, Virtual Available: 404.969 GB, Virtual Used: 098.227 GB, Swap Used: 015.686 GB
      Loaded batch of size (22, 100, 1000, 720)
      Loading /mnt/zortex-subjects/AH012/2024-08-10/5/2024-08-10_5_AH012_2P_00001_00023.tif
   Completed rigid reg on batch in :6.710351467132568s
   Shifted the mov in: 1.9412002563476562s
         After all GPU Batches:T

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

In [45]:
mov_full = job.get_registered_movie('registered_fused_data', 'fused')
im_full = mov_full[:,:200].mean(axis=1).compute()

In [46]:
# tfu.show_tif(im_full[0])

In [47]:
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy_um'] = 200
job.params['cell_filt_type'] = 'gaussian'
job.params['cell_filt_xy_um'] = 5
job.params['sdnorm_exp'] = 0.85
job.params['intensity_thresh'] = 0.05
job.params['standard_vmap'] = True
job.params['t_batch_size'] = 1000
job.params['n_proc_corr'] = 64
job.params['n_proc'] = 32
job.params['detection_timebin'] = 6

In [48]:
%%time
corr_map = job.calculate_corr_map()

   Created dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/corrmap with tag corrmap
      Updating self.dirs tag corrmap
   Created dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/mov_sub with tag mov_sub
      Updating self.dirs tag mov_sub
   Saved a copy of params at /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/corrmap
   Updated main params file
   Computing correlation map of movie with 53754 frames, volume shape: 22, 1040, 741
      Running batch 1 of 54
      Binning with timebin of size 06
         Timer batch_timebin completed in 0.002 sec


<timed exec>:1: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')


         Timer prep completed in 79.173 sec
         Timer batch_setup completed in 2.897 sec
         Timer batch_edgecrop completed in 0.225 sec
         Timer accum_meanmeax completed in 2.326 sec
         Timer batch_rolling_mean_filt completed in 2.137 sec
         Timer batch_accum_sdmov completed in 6.730 sec
         Timer batch_norm_sdmov completed in 1.120 sec
         Loading movie of size (166, 22, 1040, 741) into shared memory
         Timer dtu_shmem completed in 8.840 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 15.839 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 2.985 sec
         Timer dtu_cleanup completed in 4.746 sec
         Timer batch_filt_reduce completed in 32.412 sec
         Timer batch_accum_vmap completed in 0.069 sec
         Timer batch completed in 50.605 sec
         Timer save completed in 7.482 sec
      Running batch 2 of 54
      Binni

<timed exec>:1: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')


         Timer prep completed in 62.890 sec
         Timer batch_setup completed in 2.963 sec
         Timer batch_edgecrop completed in 0.331 sec
         Timer accum_meanmeax completed in 2.405 sec
         Timer batch_rolling_mean_filt completed in 2.144 sec
         Timer batch_accum_sdmov completed in 6.747 sec
         Timer batch_norm_sdmov completed in 1.108 sec
         Loading movie of size (166, 22, 1040, 741) into shared memory
         Timer dtu_shmem completed in 8.735 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 14.830 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 3.011 sec
         Timer dtu_cleanup completed in 4.939 sec
         Timer batch_filt_reduce completed in 31.517 sec
         Timer batch_accum_vmap completed in 0.069 sec
         Timer batch completed in 50.431 sec
         Timer save completed in 7.480 sec
      Running batch 3 of 54
      Binni

...

         Timer prep completed in 85.112 sec
         Timer batch_setup completed in 2.959 sec
         Timer batch_edgecrop completed in 0.247 sec
         Timer accum_meanmeax completed in 2.337 sec
         Timer batch_rolling_mean_filt completed in 2.165 sec
         Timer batch_accum_sdmov completed in 6.756 sec
         Timer batch_norm_sdmov completed in 1.110 sec
         Loading movie of size (166, 22, 1040, 741) into shared memory
         Timer dtu_shmem completed in 8.694 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 14.636 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 3.028 sec
         Timer dtu_cleanup completed in 4.233 sec
         Timer batch_filt_reduce completed in 30.593 sec
         Timer batch_accum_vmap completed in 0.070 sec
         Timer batch completed in 49.273 sec
         Timer save completed in 7.504 sec
      Running batch 53 of 54
      Binn

<timed exec>:1: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')


         Timer prep completed in 156.694 sec
         Timer batch_setup completed in 2.965 sec
         Timer batch_edgecrop completed in 0.277 sec
         Timer accum_meanmeax completed in 2.339 sec
         Timer batch_rolling_mean_filt completed in 2.157 sec
         Timer batch_accum_sdmov completed in 6.756 sec
         Timer batch_norm_sdmov completed in 1.112 sec
         Loading movie of size (166, 22, 1040, 741) into shared memory
         Timer dtu_shmem completed in 8.735 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 15.260 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 3.035 sec
         Timer dtu_cleanup completed in 4.341 sec
         Timer batch_filt_reduce completed in 31.373 sec
         Timer batch_accum_vmap completed in 0.071 sec
         Timer batch completed in 50.142 sec
         Timer save completed in 7.547 sec
      Running batch 54 of 54
      Bin

<timed exec>:1: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')


         Timer prep completed in 16.352 sec
         Timer batch_setup completed in 2.223 sec
         Timer batch_edgecrop completed in 0.171 sec
         Timer accum_meanmeax completed in 1.798 sec
         Timer batch_rolling_mean_filt completed in 1.632 sec
         Timer batch_accum_sdmov completed in 5.104 sec
         Timer batch_norm_sdmov completed in 0.845 sec
         Loading movie of size (125, 22, 1040, 741) into shared memory
         Timer dtu_shmem completed in 6.482 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 13.546 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 2.286 sec
         Timer dtu_cleanup completed in 3.088 sec
         Timer batch_filt_reduce completed in 25.404 sec
         Timer batch_accum_vmap completed in 0.069 sec
         Timer batch completed in 40.185 sec
         Timer save completed in 5.613 sec
CPU times: user 1h 10min 59s, sys: 21h 

In [49]:
corr_map = job.load_corr_map_results()['vmap']
corr_map_thresh = ext.thresh_mask_corr_map(corr_map, thresh_window_size_pix = 51, corrmap_thresh_pct = 50)

In [50]:
%%time
# 30min14sec
tic = time.time()
job.params['peak_thresh'] = 0.02

# when extending an ROI, compare its activity to its neighboring pixels
# in frames where the fluorescence is above this percentile
job.params['percentile'] = 99.0

job.params['extend_thresh'] = 0.05
job.params['activity_thresh'] = 5.0
job.params['max_iter'] = 1000
job.params['ext_subtract_iters'] = 3
job.params['n_proc_detect'] = 64
job.params['segmentation_timebin'] = 1
job.params['patch_size_xy'] = (250, 250)
# 4min46 for 5bin and 4 patches
# corr_map = job.load_corr_map_results()['vmap']
job.segment_rois(vmap=corr_map_thresh,)
toc = time.time()

      Found dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/segmentation
      Updating self.dirs tag segmentation
   Saved a copy of params at /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/segmentation
   Updated main params file
      Found dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/rois
      Updating self.dirs tag rois
   Saving results to /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/segmentation and /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/rois 
dict_keys(['max_img', 'mean_img', 'vmap', 'all_params'])
   Detecting from patch 1 / 20
      Found dir /mnt/md0/runs/s3d-AH012_2024-08-10_1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-19-20-21/segmentation/patch-0000
         Loading 22.85 GB movie to memory, shape: (8923, 22, 250, 250) 
         Loaded
         Loading movie patch

/home/ali/packages/s2p-lbm/suite3d/extension.py:598: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms


         Added cell 641 at 18, 246, 626, peak: 0.055, thresh: 5.000, 235 frames, 15 pixels
         Added cell 642 at 15, 243, 625, peak: 0.054, thresh: 5.000, 159 frames, 22 pixels
         Added cell 643 at 17, 266, 641, peak: 0.049, thresh: 5.000, 236 frames, 13 pixels
         Added cell 644 at 18, 326, 688, peak: 0.048, thresh: 5.000, 185 frames, 10 pixels
         Added cell 645 at 17, 306, 635, peak: 0.047, thresh: 5.000, 200 frames, 9 pixels
         Added cell 646 at 02, 227, 719, peak: 0.046, thresh: 5.000, 169 frames, 9 pixels
         Added cell 647 at 03, 383, 709, peak: 0.043, thresh: 5.000, 174 frames, 9 pixels
         Added cell 648 at 12, 240, 677, peak: 0.041, thresh: 5.000, 160 frames, 9 pixels
         Added cell 649 at 16, 365, 544, peak: 0.040, thresh: 5.000, 312 frames, 22 pixels
         Added cell 650 at 18, 289, 654, peak: 0.038, thresh: 5.000, 149 frames, 9 pixels
         Added cell 651 at 12, 227, 622, peak: 0.038, thresh: 5.000, 190 frames, 10 pixels
    

/home/ali/packages/s2p-lbm/suite3d/extension.py:598: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms
/home/ali/packages/s2p-lbm/suite3d/extension.py:598: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms
/home/ali/packages/s2p-lbm/suite3d/extension.py:598: RuntimeWarning: invalid value encountered in divide
  frac_candidate_variance_on_roi = candidate_proj_on_roi / candidate_norms


         Added cell 705 at 16, 294, 630, peak: 0.043, thresh: 5.000, 192 frames, 10 pixels
         Added cell 706 at 04, 231, 706, peak: 0.037, thresh: 5.000, 163 frames, 9 pixels
         Added cell 707 at 02, 372, 705, peak: 0.036, thresh: 5.000, 134 frames, 9 pixels
         Added cell 708 at 14, 369, 551, peak: 0.035, thresh: 5.000, 197 frames, 11 pixels
         Added cell 709 at 03, 395, 702, peak: 0.034, thresh: 5.000, 161 frames, 9 pixels
         Added cell 710 at 11, 246, 686, peak: 0.034, thresh: 5.000, 129 frames, 9 pixels
         Added cell 711 at 20, 246, 625, peak: 0.033, thresh: 5.000, 198 frames, 9 pixels
         Added cell 712 at 17, 370, 537, peak: 0.032, thresh: 5.000, 197 frames, 11 pixels
         Added cell 713 at 10, 357, 687, peak: 0.032, thresh: 5.000, 175 frames, 9 pixels
         Added cell 714 at 08, 311, 579, peak: 0.032, thresh: 5.000, 170 frames, 9 pixels
         Added cell 715 at 19, 325, 680, peak: 0.032, thresh: 5.000, 176 frames, 11 pixels
      

In [ ]:
job.compute_npil_masks(stats_dir = job.dirs['rois'])



traces = job.extract_and_deconvolve(stats_dir=job.dirs['rois'])

job.export_results('/mnt/zeytin-f1/s3d-results/',result_dir_name='rois')